In [1]:
import pandas as pd
import numpy as np
import geonamescache
import pycountry
import difflib
import warnings
import re

from countryinfo import CountryInfo
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [2]:
df = pd.read_csv("../data/3_shark_index_filas.csv")

In [3]:
df.head()

,Case Number,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,Final_year,Final_month,Final_day
0,Attack-6286,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0,2018,6,25
1,Attack-6285,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0,2018,6,18
2,Attack-6284,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0,2018,6,9
3,Attack-6283,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0,2018,6,8
4,Attack-6282,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0,2018,6,4


Ahora que tenemos nuestro propio índice y la columna Type corrediga pasamos, en este notebook, a enfrentarnos a las columnas Country y Area. El objetivo es unificar lo máximo posible a través de la corrección de erratas y nulos. Además también se buscará crear una nueva columna para clasificar los ataques por continente

Empezaremos con una exploración de los datos para luego pasar a la corrección de los mismos.

In [4]:
def corregir_geo(nombre_pais):
    try:
        # Intenta encontrar una coincidencia exacta en pycountry
        pais = pycountry.countries.get(name=nombre_pais)
        if pais:
            return pais.name  # Si se encuentra una coincidencia exacta, devuelve el nombre estándar
        else:
            # Si no se encuentra una coincidencia exacta, intenta encontrar una coincidencia cercana usando difflib
            nombres_estandar = [country.name for country in pycountry.countries]
            coincidencias_cercanas = difflib.get_close_matches(nombre_pais, nombres_estandar, n=1)
            if coincidencias_cercanas:
                return coincidencias_cercanas[0]  # Devuelve la coincidencia más cercana encontrada
            else:
                return nombre_pais  # Si no se encuentra ninguna coincidencia, devuelve el nombre original
    except LookupError:
        return nombre_pais  # Devuelve el nombre original en caso de errores

Vemos que varias filas que tienen nulos en su categoría Country también tienen en Area y Location. Decidimos eliminarlas debido a la falta de concreción geográfica del ataque.

In [5]:
len(df.Country.unique())

213

In [6]:
df.Country.replace("USA","United States", inplace=True)

In [7]:
df.Country = df.Country.apply(corregir_geo)

In [8]:
df = df.dropna(subset=["Country"])

In [9]:
len(df.Country.unique())

209

Ahora que se ha corregido la columna Country pasamos a hacer lo mismo para las columnas Area y Location.

In [10]:
print("Before:" ,len(df.Area.unique()))
print("After:", len(df.Location.unique()))

df.Area = df.Area.apply(corregir_geo)
df.Location = df.Location.apply(corregir_geo)

print("Before:", len(df.Area.unique()))
print("After:", len(df.Location.unique()))

Before: 812
After: 4090
Before: 665
After: 3760


Vemos que la función tiene gran éxito corrigiendo las columnas Area y Location, unificando casi 150 y 330 categorias, respectivamente. A continuación veremos los nulos de estas columnas.

In [11]:
len(df[df.Area.isnull()])

416

In [12]:
len(df[df.Location.isnull()])

497

Vemos que hay demasiados registros cómo para analizarlos a mano. Trataremos de arreglar la columna Area a través de la localización. A continuación revisamos las columnas donde "Area" sea null y "Location" contenga un elemento.

In [13]:
df_area = df[df.Area.isnull() & ~df.Location.isnull()]

In [14]:
len(df_area.Location)

208

La longitud continua siendo demasiado grande como para poder determinar el Area a través del Location. Por tanto decidimos asignar el valor "Unknown" a los elementos nulos

In [15]:
df['Location'].fillna("Unknown", inplace=True)

In [23]:
df['Area'].fillna("Unknown", inplace=True)

In [27]:
df.to_csv("../data/4_shark_geography.csv",index=False)